# Battle of Neighbourhoods - Week 1
This notebook consist of two parts.

## 1 Introduction/Business Problem
### 1.a Discussion of the business problem and the audience who would be interested in this project.
## 2 Data Section
### 2.a What data is used?
### 2.b Importing Libraries
### 2.c Credentials and Core location
### 2.d Search for hotel & restaurant within 1 KM
### 2.e Location of Hotels

## 1. Introduction/Business Problem

#### Discussion of the business problem and the audience who would be interested in this project. 

Douala  is the largest city in Cameroon (Central Africa) and its economic capital. It is also the capital of Cameroon's Littoral Region. Home to Central Africa's largest port and its major international airport, Douala International Airport (DLA), it is the commercial and economic capital of Cameroon and the entire CEMAC region comprising Gabon, Congo, Chad, Equatorial Guinea, Central African Republic and Cameroon. Consequently, it handles most of the country's major exports, such as oil, cocoa and coffee, timber, metals and fruits. As from 2018, the city and its surrounding area had an estimated population of 2,768,400. The city sits on the estuary of Wouri River and its climate is tropical.

#### Opening of Hotel/Restaurant Shop Coming down to business problem,
I would like to open a hotel/restaurant near beach side.As it is a famous tourist spot,there is already lots of attention towards it.I know there will be many competitors in terms of hotel and restauramt.But keeping them in mind,i need to locate my hotel in place where more people are attracted and comfortable for a stay and a good meal.I want to bring foreign and local peoples attention towards my new hotel.I would like to flavour my retaurant recipe with Italian,American,typical cameroonian delicacies to grab their taste.

The challenge is to find a suitable location for opening a new hotel / restaurant attracted to all local and foreign people in the centre of all famous venues.

Expected / Interested Audience 75% local and 25% foreign people visit Douala once in a year.Some people stay for couple of days or more.Also they find some place for hangout or a good meal.Their main focus might be belonging to stay somewhere near to reach venues.Apart from these set of people,students and working professionals are common audience here.So we may need to fascinate them all.

## 2.Data section
### 2.a What data is used?
We will be completely working on Foursquare data to explore and try to locate our new hotel where more venues like church, sports centers, beach, museums,shopping centers, memorials that are present nearby.

How will we be solving using this data?

We will looking for midpoint area of venues to locate our new hotel.Before that our major focus will be on all venues present in and around the core place of Douala.

Just a heads up on how many hotels are distributed now around Douala.We will perform some EDA on hotels & restaurants present in the tourist spot.On furthur notebook we will use Foursquare data to determine other venues as well.

### 2.b Importing Libraries

In [19]:
# Import libraries
import numpy as np # data in a vectorized manner manipulation
import pandas as pd # data analsysis
import requests # HTTP library
from bs4 import BeautifulSoup # scraping library

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import json # JSON files manipulation

from sklearn.cluster import KMeans # clustering algorithm

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

#!conda install -c conda-forge geopy --yes 
#from geopy.geocoders import Nominatim

#!conda install -c conda-forge folium=0.5.0 --yes
#import folium # map rendering library

print("*********   Loaded library     ***********")

*********   Loaded library     ***********


### 2.c Credentials and Core location

In [5]:
CLIENT_ID = 'ZIE5HF11UKWBNY4AXE452PZPMLKP1ODRGF44BPKAW2GBBIU2' # your Foursquare ID
CLIENT_SECRET = 'DGFZTAC1FE5IT0WFQYWCJBACB2O2EGGXPBQRKA52X1WHDARP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

address = "Douala, Cameroon"

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude =location.latitude   #4.0511 # location.latitude 
longitude =location.longitude #9.7679 # location.longitude # 

dla='Douala location : {},{}'.format(latitude,longitude)
print(dla)

Douala location : 4.0429408,9.706203


### 2.d Search for hotel & restaurant within 1 KM

In [6]:
# Quering for hotel & restaurant

search_query = 'hotel'
search_query_res = 'restaurant'

radius = 2000
url_hotel = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url_restaurant = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query_res, radius, LIMIT)
#url

#### Send the GET Request of hotel & restaurants and examine the results

In [7]:
results_hotel = requests.get(url_hotel).json()
results_restaurant = requests.get(url_restaurant).json()
#results_hotel

#### Get relevant part of JSON and transform it into a pandas dataframe

In [8]:
# assign relevant part of JSON to venues
venues_hotel = results_hotel['response']['venues']
venues_restaurant = results_restaurant['response']['venues']

# tranform venues into a dataframe and merging both data
dataframe_hotel = json_normalize(venues_hotel)
dataframe_restaurant = json_normalize(venues_restaurant)

dataframe = pd.concat([dataframe_hotel,dataframe_restaurant])

print("There are {} restaurants and hotels at Douala".format(dataframe.shape[0]))

There are 80 restaurants and hotels at Douala


#### Define information of interest and filter dataframe

In [21]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

    
# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

  
# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

#dataframe_filtered
hotels_df=dataframe_filtered[['name','categories','distance','lat','lng','id']]
hotels_df.head(5)

,name,categories,distance,lat,lng,id
0,Hôtel Douala Bar,Hotel,827,4.046795,9.699835,5156270ce4b00bf3a1bf5366
1,Hotel Lewat,Hotel,1617,4.057222,9.708896,50aea659e4b0f706cec130d2
2,Hotel La Falaise,Hotel,1505,4.046752,9.693191,4f0510bf77c8f87cd11c7123
3,Hôtel Beau Séjour,Resort,1578,4.050212,9.694002,52a8b0ec498ec9879bc84f5b
4,Hotel Noubou,Boarding House,241,4.045108,9.706340,58a44be704f4d741ec27c969


In [22]:
hotels_df.tail(5)

,name,categories,distance,lat,lng,id
45,shinlong restaurant chinoix bonapriso,Chinese Restaurant,2119,4.024795,9.700438,5467be8b498eb46210dfbf7f
46,la cour Restaurant,American Restaurant,2316,4.022279,9.703748,52fe162111d220fa2f63e0f8
47,Africa Lounge Restaurant,Restaurant,2386,4.022019,9.701495,58ee2548e309e128640b36dc
48,Harda West End,African Restaurant,1513,4.055093,9.700096,53469593498edd8d3cf21938
49,La Formule Provençale,African Restaurant,2163,4.039007,9.687125,501bb7e5e4b01e9e7186e7ab


#### 2.e Location of Hotels

In [18]:
hotels_map = folium.Map(location=[latitude, longitude], zoom_start=10) # generate map centred around the Kanyakumari

# add a red circle marker to represent the core location of Douala
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Douala',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
    ).add_to(hotels_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(hotels_df.lat, hotels_df.lng, hotels_df.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(hotels_map)

# display map
#hotels_map

## More to come in next notebook- "Battle of Neighbourhoods - Final"